In [7]:
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler

from fairness_checker import *

In [8]:
# To load the model back
with open('./model/german_synth.pkl', 'rb') as f:
    model = pickle.load(f)

# XGB

In [9]:
with open('./model/german_XGB.pkl', 'rb') as f:
    clf = pickle.load(f)

In [10]:
class model_wrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, file):
        df = pd.read_csv(file)
        X = self.preprocessing(df)

        return self.model.predict(X.to_numpy())

    def preprocessing(self, df_credit):

        fixed_columns = [
            'Purpose_1', 'Purpose_2', 'Purpose_3', 'Purpose_4', 'Purpose_5', 'Purpose_6', 'Purpose_7', 'Sex_1', 'Housing_1', 'Housing_2', 'Savings_1', 'Savings_2', 'Savings_3', 'Savings_4', 'Risk_1', 'Check_1', 'Check_2', 'Check_3', 'Age_cat_Young', 'Age_cat_Adult', 'Age_cat_Senior'
        ]

        # print("BEFORE")
        # print(df_credit.columns.tolist())
        # print()

        #Purpose to Dummies Variable
        df_credit = df_credit.merge(pd.get_dummies(df_credit.Purpose, drop_first=True, prefix='Purpose'), left_index=True, right_index=True)
        #Sex feature in dummies
        df_credit = df_credit.merge(pd.get_dummies(df_credit.Sex, drop_first=True, prefix='Sex'), left_index=True, right_index=True)
        # Housing get dummies
        df_credit = df_credit.merge(pd.get_dummies(df_credit.Housing, drop_first=True, prefix='Housing'), left_index=True, right_index=True)
        # Housing get Saving Accounts
        df_credit = df_credit.merge(pd.get_dummies(df_credit["Saving accounts"], drop_first=True, prefix='Savings'), left_index=True, right_index=True)
        # Housing get Risk
        df_credit = df_credit.merge(pd.get_dummies(df_credit.Risk, prefix='Risk', drop_first=True), left_index=True, right_index=True)
        # Housing get Checking Account
        df_credit = df_credit.merge(pd.get_dummies(df_credit["Checking account"], drop_first=True, prefix='Check'), left_index=True, right_index=True)

        interval = (18, 25, 35, 60, 120)
        cats = ['Student', 'Young', 'Adult', 'Senior']
        df_credit["Age_cat"] = pd.cut(df_credit.Age, interval, labels=cats)

        # Housing get Age categorical
        df_credit = df_credit.merge(pd.get_dummies(df_credit["Age_cat"], drop_first=True, prefix='Age_cat'), left_index=True, right_index=True)

        for col in fixed_columns:
            if col not in df_credit.columns:
                # print("Added col", col)
                df_credit[col] = 0

        #Excluding the missing columns
        del df_credit["Saving accounts"]
        del df_credit["Checking account"]
        del df_credit["Purpose"]
        del df_credit["Sex"]
        del df_credit["Housing"]
        del df_credit["Age_cat"]
        del df_credit["Risk"]
        # del df_credit['Risk_0']

        df_credit['Credit amount'] = np.log(df_credit['Credit amount'])

        df_credit = df_credit.drop('Risk_1', axis=1)



        # print("AFTER")
        # print(df_credit.columns.tolist())
        # print()

        # current_features = df_credit.columns.tolist()

        # missing_features = set(current_features) - set(trained_features)
        # print("Missing features:", missing_features)

        return df_credit

trained = model_wrapper(clf)

In [11]:
# this takes 10 minutes
iter = 10

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for i in range(iter):
    if i % 5 == 0:
        print("iter:", i)
    synth = model.synthetic_data(rows=1000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['Sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['Risk'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['Job'] == x), ('1',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

iter: 0
iter: 5


In [12]:
og_disparate_impact = 0
og_demographic_parity = 0
og_equalized_odds_1, og_equalized_odds_2 = 0, 0
og_equal_opportunity = 0
og_accuracy_eqaulity = 0
og_predictive_parity = 0
og_equal_calibration = 0
og_conditional_statistical_parity = 0
og_predictive_equality = 0
og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2 = 0, 0
og_positive_balance = 0
og_negative_balance = 0
og_mean_difference = 0

cog = fairness_model_checker("./data/german_processed.csv", verbose=False)

privileged_predicate = lambda row: row['Sex'] != '0'
positive_predicate = lambda Y: Y == 1
truth_predicate = lambda row: row['Risk'] == '1'

og_disparate_impact = cog.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True)
og_demographic_parity = cog.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True)
(og_equalized_odds_1, og_equalized_odds_2) = cog.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_equal_opportunity = cog.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_accuracy_eqaulity = cog.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_predictive_parity = cog.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# equal_calibration (n/a)
og_conditional_statistical_parity = cog.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['Job'] == x), ('1',), value=True)
og_predictive_equality = cog.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
(og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2) = cog.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# positive_balance (n/a)
# negative_balance (n/a)
og_mean_difference = cog.mean_difference(0.2, trained, privileged_predicate, positive_predicate, value=True)

/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/anaconda3/envs/mbi/lib/python

In [13]:
print("name               ", "og          ", "synth       ", "diff        ", "std         ", "ratio")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", f"{np.std(np.array(demographic_parity)):.10f}", f"{og_demographic_parity / np.mean(np.array(demographic_parity)):.10f}")
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", f"{np.std(np.array(accuracy_eqaulity)):.10f}", f"{og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity)):.10f}")
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", f"{np.std(np.array(equalized_odds_1)):.10f}", f"{og_equalized_odds_1 / np.mean(np.array(equalized_odds_1)):.10f}")
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", f"{np.std(np.array(equalized_odds_2)):.10f}", f"{og_equalized_odds_2 / np.mean(np.array(equalized_odds_2)):.10f}")
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}")
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", f"{np.std(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}")
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", f"{np.std(np.array(mean_difference)):.10f}", f"{og_mean_difference / np.mean(np.array(mean_difference)):.10f}")

print(
"sum of diff:",
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

print(
"avg of ratio:",
(og_demographic_parity / np.mean(np.array(demographic_parity))+
og_accuracy_eqaulity / np.mean(np.array(accuracy_eqaulity))+
og_equalized_odds_1 / np.mean(np.array(equalized_odds_1))+
og_equalized_odds_2 / np.mean(np.array(equalized_odds_2))+
og_conditional_use_accuracy_equality_1 / np.mean(np.array(conditional_use_accuracy_equality_1))+
og_conditional_use_accuracy_equality_2 / np.mean(np.array(conditional_use_accuracy_equality_2))+
og_mean_difference / np.mean(np.array(mean_difference))) / 7
)

name                og           synth        diff         std          ratio
demographic_parity  0.0148200094 0.1359467888 0.1211267794 0.0112536747 0.1090133094
accuracy_eqaulity   0.0543712015 0.2217233727 0.1673521712 0.0133237331 0.2452208842
equalized_odds_1    0.0478889476 0.0785245730 0.0306356254 0.0179392661 0.6098593828
equalized_odds_2    0.0691931126 0.1998193924 0.1306262798 0.1135920407 0.3462782654
accuracy_equality_1 0.1205673759 0.1122169166 0.0083504593 0.0375831250 1.0744135511
accuracy_equality_2 0.0278110765 0.1392699054 0.1114588289 0.0688667636 0.1996919320
mean_difference     0.2570827489 0.1723787715 0.0847039775 0.0198112074 1.4913828815
sum of diff: 0.6542541213907759
avg of ratio: 0.5822657437755332
